# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure gmaps with API key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

output_data_file = "../output_data/cities.csv"
city_data = pd.read_csv(output_data_file)
# city_data.dropna(inplace = True)
city_data.head()
# city_data_df = pd.DataFrame(city_data, columns=['City', 'Lat', 'Lng', 'Max Temp', 'Current Description', 'Humidity', 'Cloudiness', 'Wind Speed', 'Country'])


# city_data_df.head(25)

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [36]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# locations=city_data[['Lat', 'Lng']].astype(float)

# hum_max=city_data['Humidity'].max()



In [39]:
locations = city_data[["Lat", "Lng"]]
humidity = city_data["Humidity"]

fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=300,
                                 point_radius=5)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Narrow down the DataFrame to find your ideal weather condition. For example:
A max temperature lower than 80 degrees but higher than 70

Wind speed less than 10 mph

Zero cloudiness

In [42]:
narrow_city = city_data.loc[(city_data['Max Temp']< 80) & (city_data['Max Temp'] >70) & (city_data['Wind Speed'] <10) & (city_data['Cloudiness'] !=0)].dropna()
narrow_city.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
12,12,avarua,100,CK,1558378756,94,-21.21,-159.78,71.60,2.24
25,25,kantang,20,TH,1558378759,94,7.42,99.52,78.80,6.93
44,44,puerto ayora,26,EC,1558378762,86,-0.74,-90.35,79.00,5.99
59,59,bontang,54,ID,1558378765,99,0.12,117.47,74.22,2.62
87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [43]:
hotel_df = pd.DataFrame()
name_hotel = pd.DataFrame(narrow_city)
hotel_df['City'] =name_hotel['City']
hotel_df['Country'] = name_hotel['Country']
hotel_df['Lat'] = name_hotel['Lat']
hotel_df['Lng'] = name_hotel['Lng']

hotel_df.head()

,City,Country,Lat,Lng
12,avarua,CK,-21.21,-159.78
25,kantang,TH,7.42,99.52
44,puerto ayora,EC,-0.74,-90.35
59,bontang,ID,0.12,117.47
87,ambilobe,MG,-13.19,49.05


In [44]:
# Set parameters to serach for a hotel
parameters = { "radius": 5000, "types": "lodging","key": g_key}

# Iterate through cities
for index, row in hotel_df.iterrows():
    # get lat, lng from dataframe
    latitude = row["Lat"]
    longitude = row["Lng"]

    # change location each iteration while keeping original params in place
    parameters["location"] = f"{latitude},{longitude}"
    
    # Create base url to search nearby hotel     
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request
    print(f"Retrieving results for city: {row['City']}")
    hotel_name = requests.get(base_url, params=parameters).json()
    
    # extract results
    hotel_name = hotel_name['results']
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name[0]['name']
        print(f"Closest hotel is {hotel_name[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = hotel_name[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

# Display hotel names on dataframe
hotel_df

Retrieving results for city: avarua
Closest hotel is Paradise Inn.
------------
Retrieving results for city: kantang
Closest hotel is Reng Ta Chai Boutique HomeStay.
------------
Retrieving results for city: puerto ayora
Closest hotel is Finch Bay Galapagos Hotel.
------------
Retrieving results for city: bontang
Closest hotel is Guest House Badak LNG,Bontang,Kaliantan Timur.
------------
Retrieving results for city: ambilobe
Closest hotel is Hôtel & Restaurant DIANA Ambilobe.
------------
Retrieving results for city: bella vista
Closest hotel is Hotel Río Arriba.
------------
Retrieving results for city: sorong
Closest hotel is Waigo Splash Hotel.
------------
Retrieving results for city: mbigou
Missing field/result... skipping.
------------
Retrieving results for city: hilo
Closest hotel is SCP Hilo Hotel.
------------
Retrieving results for city: porto novo
Closest hotel is Harmonia.
------------
Retrieving results for city: manado
Closest hotel is Aryaduta Hotel.
------------
Retri

,City,Country,Lat,Lng,Hotel Name
12,avarua,CK,-21.21,-159.78,Paradise Inn
25,kantang,TH,7.42,99.52,Reng Ta Chai Boutique HomeStay
44,puerto ayora,EC,-0.74,-90.35,Finch Bay Galapagos Hotel
59,bontang,ID,0.12,117.47,"Guest House Badak LNG,Bontang,Kaliantan Timur"
87,ambilobe,MG,-13.19,49.05,Hôtel & Restaurant DIANA Ambilobe
91,bella vista,AR,-28.51,-59.05,Hotel Río Arriba
100,sorong,ID,-0.86,131.25,Waigo Splash Hotel
104,mbigou,GA,-1.90,11.91,NaN
106,hilo,US,19.71,-155.08,SCP Hilo Hotel
153,porto novo,BR,-23.68,-45.44,Harmonia


In [45]:
hotel_df.count()

City          51
Country       51
Lat           51
Lng           51
Hotel Name    47
dtype: int64

In [46]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations_loc = hotel_df[["Lat", "Lng"]]

In [49]:
# Add marker layer ontop of heat map

figure = gmaps.figure()

# Create heat layer
markers = gmaps.marker_layer(locations_loc, info_box_content = hotel_info)

# Add layer

fig.add_layer(markers)

plt.savefig("vacationheatmapwithhotels.png")



# Display figure
fig




Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>